# Introduction

This notebook demonstrates how to train custom openWakeWord models using pre-defined datasets, an automated process for synthetic data generation/augmentation, and a custom training process. While not guaranteed to always produce the best performing model, the methods shown in this notebook often produce baseline models with relatively strong performance.

Manual data preparation and model training (e.g., see the [training models](training_models.ipynb) notebook) remains an option for when full control over the model development process is needed.

At a high level, the automatic training process takes advantages of several techniques to try and produce a good model, including:

- Early-stopping and checkpoint averaging (similar to [stochastic weight averaging](https://arxiv.org/abs/1803.05407)) to search for the best models found during training, according to the validation data
- Variable learning rates with cosine decay and multiple cycles
- Adaptive batch construction to focus on only high-loss examples when the model begins to converge, combined with gradient accumulation to ensure that batch sizes are still large enough for stable training
- Cycical weight schedules for negative examples to help the model reduce false-positive rates

See the contents of the `train.py` file for more details.

# Environment Setup

To begin, we'll need to install the requirements for training custom models. In particular, a relatively recent version of Pytorch and custom fork of the [piper-sample-generator](https://github.com/dscripka/piper-sample-generator) library for generating synthetic examples for the custom model.

**Important Note!** Currently, automated model training is only supported on linux systems due to the requirements of the text to speech library used for synthetic sample generation (Piper). It may be possible to use Piper on Windows/Mac systems, but that has not (yet) been tested.

In [ ]:
## Environment setup

# install piper-sample-generator (currently only supports linux systems)
!git clone https://github.com/dscripka/piper-sample-generator
!wget -O piper-sample-generator/models/en-us-libritts-high.pt 'https://github.com/rhasspy/piper-sample-generator/releases/download/v1.0.0/en-us-libritts-high.pt'
!apt-get install libespeak-ng1  # may not be required on all systems
!pip install espeak_phonemizer

# install openwakeword (full installation to support training)
!git clone https://github.com/dscripka/openwakeword
!pip install -e ./openwakeword[full]
!cd openwakeword


In [1]:
# Imports

import os
import torch
from openwakeword.data import mmap_batch_generator, generate_adversarial_texts
from openwakeword.utils import compute_features_from_generator
import sys
from pathlib import Path
import uuid
import yaml
import datasets
import scipy
from tqdm import tqdm


/home/dscripka/anaconda3/envs/openwakeword_dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download Data

When training new openWakeWord models using the automated procedure, four specific types of data are required:

1) Synthetic examples of the target word/phrase generated with text-to-speech models

2) Synthetic examples of adversarial words/phrases generated with text-to-speech models

3) Room impulse reponses and noise/background audio data to augment the synthetic examples and make them more realistic

4) Generic "negative" audio data that is very unlikely to contain examples of the target word/phrase in the context where the model should detect it. This data can be the original audio data, or precomputed openWakeWord features ready for model training.

5) Validation data to use for early-stopping when training the model.

For the purposes of this notebook, all five of these sources will either be generated manually or can be obtained from HuggingFace thanks to their excellent `datasets` library and extremely generous hosting policy. Also note that while only a portion of some datasets are downloaded, for the best possible performance it is recommended to download the entire dataset and keep a local copy for future training runs.

In [15]:
# Download room impulse responses collected by MIT
# https://mcdermottlab.mit.edu/Reverb/IR_Survey.html

output_dir = "./mit_rirs"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
rir_dataset = datasets.load_dataset("davidscripka/MIT_environmental_impulse_responses", split="train", streaming=True)

for row in tqdm(rir_dataset):
    name = row['audio']['path'].split('/')[-1]
    scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, row['audio']['array'])


In [ ]:
## Download noise and background audio

# FSD50k Noise Dataset (warning, this can take 5 minutes to prepare when streaming)
# https://zenodo.org/record/4060432
output_dir = "./fsd50k"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
fsd50k_dataset = datasets.load_dataset("Fhrozen/FSD50k", split="validation", streaming=True)  # ~40,000 files in this split
fsd50k_dataset = iter(fsd50k_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000)))

n_total = 500  # use only 500 clips for this example notebook, recommend increasing for full-scale training
for i in tqdm(range(n_total)):
    row = next(fsd50k_dataset)
    name = row['audio']['path'].split('/')[-1]
    scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, row['audio']['array'])
    i += 1
    if i == n_total:
        break

# Free Music Archive dataset
# https://github.com/mdeff/fma

output_dir = "./fma"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
fma_dataset = datasets.load_dataset("rudraml/fma", name="small", split="train", streaming=True)
fma_dataset = iter(fma_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000)))

n_hours = 1  # use only 1 hour of clips for this example notebook, recommend increasing for full-scale training
for i in tqdm(range(n_hours*3600//30)):  # this works because the FMA dataset is all 30 second clips
    row = next(fma_dataset)
    name = row['audio']['path'].split('/')[-1]
    scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, row['audio']['array'])
    i += 1
    if i == n_hours*3600//30:
        break


In [ ]:
# Download pre-computed openWakeWord features for training and validation

# training set (~2,000 hours)
!wget https://huggingface.co/datasets/davidscripka/openwakeword_features/blob/main/openwakeword_features_ACAV100M_2000_hrs_16bit.npy

# validation set for false positive rate estimation (~11 hours)
!wget https://huggingface.co/datasets/davidscripka/openwakeword_features/blob/main/validation_set_features.npy

# Define Training Configuration

For automated model training openWakeWord uses a specially designed training script and a [YAML](https://yaml.org/) configuration file that defines all of the information required for training a new wake word/phrase detection model.

It is strongly recommended that you review [the example config file](../examples/custom_model.yml), as each value is fully documented there. For the purposes of this notebook, we'll read in the YAML file to modify certain configuration parameters before saving a new YAML file for training our example model. Specifically:

- We'll train a detection model for the phrase "hey sebastian"
- We'll only generate 5,000 positive and negative examples (to save on time for this example)
- We'll only generate 1,000 validation positive and negative examples for early stopping (again to save time)
- The model will only be trained for 10,000 steps (larger datasets will benefit from longer training)
- We'll reduce the target metrics to account for the small dataset size and limited training.

On the topic of target metrics, there are *not* specific guidelines about what these metrics should be in practice, and you will need to conduct testing in your target deployment environment to establish good thresholds. However, from very limited testing the default values in the config file (accuracy >= 0.7, recall >= 0.5, false-positive rate <= 0.2 per hour) seem to produce model with reasonable performance.


In [89]:
# Load default YAML config file for training
config = yaml.load(open("openwakeword/examples/custom_model.yml", 'r').read(), yaml.Loader)
config

{'model_name': 'my_model',
 'target_phrase': ['hey jarvis'],
 'total_length': 32000,
 'custom_negative_phrases': [],
 'n_samples': 10000,
 'n_samples_val': 2000,
 'tts_batch_size': 50,
 'augmentation_batch_size': 16,
 'piper_sample_generator_path': './piper-sample-generator',
 'output_dir': './generated_data',
 'rir_paths': ['./mit_rirs'],
 'background_paths': ['./background_clips'],
 'false_positive_validation_data_path': './validation_set_features.npy',
 'augmentation_rounds': 1,
 'feature_data_files': {'ACAV100M_sample': './openwakeword_features_ACAV100M_2000_hrs_16bit.npy'},
 'batch_n_per_class': {'ACAV100M_sample': 1024,
  'adversarial_negative': 50,
  'positive': 50},
 'model_type': 'dnn',
 'layer_size': 32,
 'steps': 100000,
 'max_negative_weight': 1500,
 'target_accuracy': 0.7,
 'target_recall': 0.5,
 'target_false_positives_per_hour': 0.2}

In [86]:
# Modify values in the config and save a new version

config["target_phrase"] = ["hey sebastian"]
config["n_samples"] = 1000
config["n_samples_val"] = 1000
config["steps"] = 10000
config["target_accuracy"] = 0.6
config["target_recall"] = 0.25

config["background_paths"] = []#['./fsd50k', './fma']
config["false_positive_validation_data_path"] = "val_set_features.npy"
config["feature_data_files"] = {"ACAV100M_sample": "openwakeword_features_ACAV100M_2000_hrs_16bit.npy"}

with open('my_model.yaml', 'w') as file:
    documents = yaml.dump(config, file)

# Train the Model

With the data downloaded and training configuration set, we can now start training the model. We'll do this in parts to better illustrate the sequence, but you can also execute every step at once for a fully automated process.

In [15]:
# Step 1: Generate synthetic clips
# For the number of clips we are using, this should take ~10 minutes on a free Google Colab instance
# If generation fails, you can simply run this command again as it will continue generating until the
# number of files meets the targets specified in the config file

!{sys.executable} openwakeword/openwakeword/train.py --training_config my_model.yaml --generate_clips

INFO:root:Generating positive clips for training
INFO:root:Generating positive clips for testing
INFO:root:Generating negative clips for training
INFO:root:Generating negative clips for testing


In [24]:
# Step 2: Augment the generated clips

!{sys.executable} openwakeword/openwakeword/train.py --training_config my_model.yaml --augment_clips

INFO:root:Creating augmentation generators
INFO:root:Computing openwakeword features for generated samples
Computing features: 100%|███████████████████████| 62/62 [00:05<00:00, 10.58it/s]
Trimming empty rows: 1it [00:00,  7.99it/s]
Computing features: 100%|███████████████████████| 62/62 [00:05<00:00, 10.45it/s]
Trimming empty rows: 1it [00:00,  8.07it/s]
Computing features: 100%|███████████████████████| 62/62 [00:05<00:00, 10.77it/s]
Trimming empty rows: 1it [00:00,  8.09it/s]
Computing features: 100%|███████████████████████| 62/62 [00:05<00:00, 10.53it/s]
Trimming empty rows: 1it [00:00,  8.07it/s]


In [87]:
# Step 3: Train model

!{sys.executable} openwakeword/openwakeword/train.py --training_config my_model.yaml --train_model

INFO:root:Starting training sequence 1...
Training:  75%|████████████████████▏      | 7496/10000 [00:45<00:14, 168.37it/s]1.3274336 0.66 0.331
INFO:root:Saving checkpoint with metrics >= to targets!
Training:  76%|█████████████████████▎      | 7621/10000 [00:49<00:28, 82.33it/s]1.3274336 0.66 0.331
INFO:root:Saving checkpoint with metrics >= to targets!
Training:  79%|██████████████████████      | 7887/10000 [00:57<00:25, 83.36it/s]0.088495575 0.607 0.216
INFO:root:Saving checkpoint with metrics >= to targets!
Training:  83%|███████████████████████▏    | 8277/10000 [01:08<00:20, 82.58it/s]0.088495575 0.605 0.212
INFO:root:Saving checkpoint with metrics >= to targets!
Training: 100%|███████████████████████████▉| 9999/10000 [01:57<00:00, 85.19it/s]
INFO:root:Starting training sequence 2...
Training:   4%|█▏                           | 41/1000.0 [00:00<00:11, 86.52it/s]1.1504425 0.658 0.33
Training:   9%|██▌                          | 89/1000.0 [00:04<00:37, 24.32it/s]1.4159292 0.6595 0.3

2023-09-04 11:11:13.859582: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
INFO:tensorflow:Assets written to: /tmp/tmpqvff79vt/tf_model/assets
2023-09-04 11:11:14.012590: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-09-04 11:11:14.012606: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-09-04 11:11:14.013126: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpqvff79vt/tf_model
2023-09-04 11:11:14.013536: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-09-04 11:11:14.013546: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpqvff79vt/tf_model
2023-09-04 11:11:14.014464: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-09-04 11:11:14